# *For this Book, keep in mind that the exercises I've tested on DataLab to get the database set up*

### WINDOW FUNCTIONS

In [3]:
SELECT * 
FROM TFL.JOURNEYS
LIMIT 10;

,month,year,days,report_date,journey_type,journeys_millions
0,2,2020,29,2020-06-30,Underground & DLR,8.041099
1,2,2020,29,2020-06-30,Bus,32.467243
2,2,2020,29,2020-06-30,Tram,0.470811
3,2,2020,29,2020-06-30,Overground,1.491890
4,2,2020,29,2020-06-30,Emirates Airline,0.004978
5,2,2020,29,2020-06-30,TfL Rail,0.700218
6,1,2020,31,2020-06-30,Underground & DLR,6.950757
7,3,2020,31,2020-06-30,Underground & DLR,15.440623
8,1,2020,31,2020-06-30,Bus,30.223736
9,3,2020,31,2020-06-30,Bus,46.959520


In [22]:
SELECT journey_type, journeys_millions,
		AVG(journeys_millions) OVER (PARTITION BY journey_type ORDER BY journey_type) AS month_journeys
FROM TFL.JOURNEYS

,journey_type,journeys_millions,month_journeys
0,Bus,32.467243,159.648679
1,Bus,30.223736,159.648679
2,Bus,46.959520,159.648679
3,Bus,188.778362,159.648679
4,Bus,171.634596,159.648679
...,...,...,...
931,Underground & DLR,92.480380,96.285042
932,Underground & DLR,118.764537,96.285042
933,Underground & DLR,104.835566,96.285042
934,Underground & DLR,102.628480,96.285042


In [21]:
SELECT journey_type, AVG(journeys_millions) AS total_sum
FROM TFL.JOURNEYS
GROUP BY journey_type
ORDER BY journey_type;

,journey_type,total_sum
0,Bus,159.648679
1,Emirates Airline,0.113052
2,Overground,11.186884
3,TfL Rail,4.329615
4,Tram,2.017243
5,Underground & DLR,96.285042


The difference between the Aggregate Functions and Window Functions is that for the first, it returns only one value based on the GROUP BY clause, for the latter, it returns the calculated value but for the current row

- Aggregate Functions: Collapse rows into a single result per group.
- Window Functions: Maintain the original number of rows, adding calculated values to each row.

The Top 5 Most Popular Window Functions and How to Use Them
- SUM()
- ROW_NUMBER()
- DENSE_RANK()
- LAG()
- MAX()

In [13]:
-- Testing ROW_NUMBER() with subquery

SELECT report_date, journeys_millions, row_numbering
FROM 
(
SELECT report_date, journeys_millions,
		ROW_NUMBER() OVER (ORDER BY report_date) AS row_numbering
FROM TFL.JOURNEYS
)
WHERE row_numbering = 1 OR row_numbering = 2 OR row_numbering = 3 OR row_numbering = 100 OR row_numbering = 200 OR row_numbering = 300 OR row_numbering = 400

,report_date,journeys_millions,row_numbering
0,2010-06-30,90.330504,1
1,2010-06-30,181.604792,2
2,2010-06-30,2.154790,3
3,2011-09-30,NaN,100
4,2013-03-31,178.122194,200
5,2014-06-30,2.288241,300
6,2015-12-31,15.753193,400


In [20]:
-- Testing ROW_NUMBER() with CTEs

WITH rows_numbered AS (
    SELECT 
        j.report_date, 
        j.journeys_millions,
        ROW_NUMBER() OVER (ORDER BY j.report_date) AS row_numbering
    FROM 
        TFL.JOURNEYS AS j
)
SELECT
	r.report_date, 
    r.journeys_millions, 
    r.row_numbering
FROM 
    rows_numbered AS r
WHERE 
    row_numbering IN (1, 2, 3, 100, 200, 300, 400);

,report_date,journeys_millions,row_numbering
0,2010-06-30,90.330504,1
1,2010-06-30,181.604792,2
2,2010-06-30,2.154790,3
3,2011-09-30,NaN,100
4,2013-03-31,178.122194,200
5,2014-06-30,2.288241,300
6,2015-12-31,15.753193,400


In [19]:
-- Testing MAX()
SELECT j.month,
		j.year,
		j.report_date,
		j.journeys_millions,
        MAX(j.journeys_millions) OVER(ORDER BY j.report_date 
                         ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING) as max_journeys
FROM TFL.JOURNEYS AS j
ORDER BY j.report_date
;

,month,year,report_date,journeys_millions,max_journeys
0,2,2010,2010-06-30,NaN,181.604792
1,3,2010,2010-06-30,NaN,NaN
2,3,2010,2010-06-30,90.038014,96.836391
3,3,2010,2010-06-30,175.883173,189.114661
4,2,2010,2010-06-30,NaN,181.604792
...,...,...,...,...,...
931,10,2022,2023-03-31,1.295158,141.602073
932,12,2022,2023-03-31,0.116734,13.364324
933,10,2022,2023-03-31,74.262699,141.285241
934,10,2022,2023-03-31,116.911790,97.599149
